In [ ]:
import pickle
import os
import sys
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

get_ipython().magic('matplotlib inline')
get_ipython().magic('reload_ext autoreload')
get_ipython().magic('autoreload 2')

# TODO
# Look at specific weird outlier points to see what's going on (especially for suit frequency patterns)
# Plot everything (aka abstract the code so I can just pass in options and it will plot stuff)
# day of week plots

# Do All Processing

In [ ]:
from temporal_processing import TimeAnalysis

data_file = '../data/newsAnchor_all_data_random.pkl'
analysis = TimeAnalysis(data_file)

In [ ]:
# analysis_type, attribute, gender_constraint=None, network_constraint=None)
constraint_list = [#(TimeAnalysis.CONSTRAINT_FEMALE, TimeAnalysis.CONSTRAINT_CNN), \
                   (TimeAnalysis.CONSTRAINT_FEMALE, TimeAnalysis.CONSTRAINT_FOX), \
                   (TimeAnalysis.CONSTRAINT_FEMALE, TimeAnalysis.CONSTRAINT_MSNBC)]
# constraint_list = [(None, TimeAnalysis.CONSTRAINT_CNN), \
#                    (None, TimeAnalysis.CONSTRAINT_FOX), \
#                    (None, TimeAnalysis.CONSTRAINT_MSNBC)]
# constraint_list = [(None, None)]
analysis.run_analysis(TimeAnalysis.TYPE_ALL_TIME_BY_WEEK, 7, constraint_list)

In [ ]:
attribs_to_run = analysis.get_attribute_list()
types_to_run = analysis.get_analysis_type_list()
genders_to_run = analysis.get_gender_constraint_list()
networks_to_run = analysis.get_network_constraint_list()

for attrib in attribs_to_run:
    print('Attrib ' + str(attrib) + '...')
    for analysis_type in types_to_run:
        print('Analysis ' + str(analysis_type) + '...')
        # no constraints
        analysis.run_analysis(analysis_type, attrib, [(None, None)])
        print('No constraints done!')
        # single constraints
        for gender in genders_to_run:
            analysis.run_analysis(analysis_type, attrib, [(gender, None)])
        for net in networks_to_run:
            analysis.run_analysis(analysis_type, attrib, [(None, net)])
        print('Single constraints done!')
        # both constraints
        for gender in genders_to_run:
            for net in networks_to_run:
                analysis.run_analysis(analysis_type, attrib, [(gender, net)])
        print('Combined constraints done!')

# Can Run Individually

In [ ]:
data_file = '../data/newsAnchor_all_data_random.pkl'
data = pickle.load(open(data_file, 'rb'), encoding='latin1')

In [ ]:
import tz_util
from datetime import datetime, timezone

network_dict = {'FOXNEWS' : 0, 'CNN' : 1, 'MSNBC' : 2}
gender_dict = {'F' : 0, 'M' : 1}

date = []
time = []
network = []
gender = []
attributes = []
for anchor in data:
    show_info = anchor[3]
    date_time = show_info[0]
    gmt = datetime(date_time[0], date_time[1], date_time[2], \
                             date_time[3], date_time[4], date_time[5], tzinfo=timezone.utc)
    et = gmt.astimezone(tz_util.Eastern)
    date.append((et.year, et.month, et.day))
    time.append((et.hour, et.minute, et.second))
    network.append(network_dict[show_info[1]])
    gender.append(gender_dict[anchor[5]])
    attributes.append(anchor[6])

In [ ]:
print(len(data))
test = 69026
print(data[test])
print(date[test])
print(time[test])
print(network[test])
print(gender[test])
print(attributes[test])

In [ ]:
clothing_pattern = ['solid', 'graphics', 'striped', 'floral', 'plaid', 'spotted']
major_color = ['black', 'white', '2+ colors', 'blue', 'gray', 'red',
                'pink', 'green', 'yellow', 'brown', 'purple', 'orange',
                'cyan', 'dark blue']
wearing_necktie = ['necktie no', 'necktie yes']
collar_presence = ['collar no', 'collar yes'] 
wearing_scarf = ['scarf no', 'scarf yes']
sleeve_length = ['long sleeve', 'short sleeve', 'no sleeve']
neckline_shape = ['round', 'folded', 'v-shape'] 
clothing_category = ['shirt', 'outerwear', 't-shirt', 'dress',
            'tank top', 'suit', 'sweater']
wearing_jacket = ['jacket no', 'jacket yes']
wearing_hat = ['hat no', 'hat yes']
wearing_glasses = ['no glasses', 'yes glasses']
multiple_layers = ['one layer', 'more layer']
necktie_color = major_color
necktie_pattern = ['solid', 'striped', 'spotted']
hair_color = ['black', 'white', 'blond', 'brown', 'gray']
hair_length = ['long', 'medium', 'short', 'bald']
network_attribs = ['FOXNEWS', 'CNN', 'MSNBC']
gender_attribs = ['Female', 'Male']

# Major Color Over Time

In [ ]:
min_year = 2015
min_month = 1
min_day = 1
months_per_year = 12
weeks_per_month = 4
days_per_month = 31
week_length = 1.*days_per_month / weeks_per_month

def calc_week_index(date):
    year, month, day = date
    if year < min_year:
        idx = 0
    else:
        idx = (year - min_year)*months_per_year*weeks_per_month + \
                      (month - min_month)*weeks_per_month + int((day - min_day) / week_length)
    return idx

def date_from_index(idx, string=False):
    year = int(1.*idx / (months_per_year*weeks_per_month)) + min_year
    month = int((idx % (months_per_year*weeks_per_month)) / weeks_per_month) + min_month
    day = int(((idx % (months_per_year*weeks_per_month)) % weeks_per_month)*week_length) + min_day
    if string:
        return str(month) + '/' + str(day) + '/' + str(year)
    else:
        return (year, month, day)
        

total_weeks = calc_week_index((2017, 12, 31)) - 1
print(total_weeks)
    
print(calc_week_index((2015, 1, 1)))
print(calc_week_index((2016, 1, 1)))

### Total Colors

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
color_count = []
for i in range(0, len(major_color)):
    color_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    week_idx = calc_week_index(date[i])
    color = attributes[i][1]
    color_count[color][week_idx] += 1
    total_count[week_idx] += 1

print(total_count[-1])

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(major_color)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    color_freq = color_count[i] / total_count
    # smooth data
    color_smooth = savgol_filter(color_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, color_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle(major_color[i].capitalize())
    plt.savefig('./results/all_' + major_color[i].replace(' ', '') + '.png')

### Women Colors

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
color_count = []
for i in range(0, len(major_color)):
    color_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if gender[i] == gender_dict['F']:
        week_idx = calc_week_index(date[i])
        color = attributes[i][1]
        color_count[color][week_idx] += 1
        total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(major_color)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    color_freq = color_count[i] / total_count
    # smooth data
    color_smooth = savgol_filter(color_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, color_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('Female - ' + major_color[i].capitalize())
    plt.savefig('./results/female_' + major_color[i].replace(' ', '') + '.png')

### Male Colors

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
color_count = []
for i in range(0, len(major_color)):
    color_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if gender[i] == gender_dict['M']:
        week_idx = calc_week_index(date[i])
        color = attributes[i][1]
        color_count[color][week_idx] += 1
        total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(major_color)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    color_freq = color_count[i] / total_count
    # smooth data
    color_smooth = savgol_filter(color_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, color_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('Male - ' + major_color[i].capitalize())
    plt.savefig('./results/male_' + major_color[i].replace(' ', '') + '.png')

### CNN

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
color_count = []
for i in range(0, len(major_color)):
    color_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if network[i] == network_dict['CNN']:
        week_idx = calc_week_index(date[i])
        color = attributes[i][1]
        color_count[color][week_idx] += 1
        total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(major_color)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    color_freq = color_count[i] / total_count
    # smooth data
    color_smooth = savgol_filter(color_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, color_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('CNN - ' + major_color[i].capitalize())
    plt.savefig('./results/cnn_' + major_color[i].replace(' ', '') + '.png')

### FOX

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
color_count = []
for i in range(0, len(major_color)):
    color_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if network[i] == network_dict['FOXNEWS']:
        week_idx = calc_week_index(date[i])
        color = attributes[i][1]
        color_count[color][week_idx] += 1
        total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(major_color)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    color_freq = color_count[i] / total_count
    # smooth data
    color_smooth = savgol_filter(color_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, color_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('FOXNEWS - ' + major_color[i].capitalize())
    plt.savefig('./results/fox_' + major_color[i].replace(' ', '') + '.png')

### MSNBC

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
color_count = []
for i in range(0, len(major_color)):
    color_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if network[i] == network_dict['MSNBC']:
        week_idx = calc_week_index(date[i])
        color = attributes[i][1]
        color_count[color][week_idx] += 1
        total_count[week_idx] += 1
        
print(total_count[-1])
print(color_count[3][-1])

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(major_color)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    color_freq = color_count[i] / total_count
    # smooth data
    color_smooth = savgol_filter(color_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, color_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('MSNBC - ' + major_color[i].capitalize())
    plt.savefig('./results/msnbc_' + major_color[i].replace(' ', '') + '.png')

### Bin by Day

In [ ]:
# structures to hold color frequency info
total_count = {}
color_count = []
for i in range(0, len(major_color)):
    color_count.append({})
    
# count frequency of each color
for i in range(0, len(data)):
    cur_date = date[i]
    color = attributes[i][1]
    if cur_date in color_count[color]:
        color_count[color][cur_date] += 1
    else:
        color_count[color][cur_date] = 1
    if cur_date in total_count:
        total_count[cur_date] += 1
    else:
        total_count[cur_date] = 1
        
# calc frequencies
color_freqs = [np.zeros((len(total_count))) for i in range(0, len(major_color))]
counted_dates = sorted(total_count.keys())
xdates = []
year_dates = []
for i, counted_date in enumerate(counted_dates):
    for j, color_freq in enumerate(color_freqs):
        if counted_date in color_count[j]:
            color_freq[i] = 1.*color_count[j][counted_date] / total_count[counted_date]
        else:
            color_freq[i] = 0.
    if i % 107 == 0:
        year, month, day = counted_date
        xdates.append(datetime.date(year, month, day).strftime('%b %y'))
    if counted_date == (2016, 1, 1) or counted_date == (2017, 1, 1):
        year_dates.append(i)
print(year_dates)
print(xdates)

In [ ]:
from scipy.signal import savgol_filter
import matplotlib.ticker as plticker

# plot
xraw = np.arange(0, len(counted_dates))
for i in range(0, len(major_color)):
    color_freq = color_freqs[i]
    # smooth data
    color_smooth = savgol_filter(color_freq, 51, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.xticks(xraw, xdates)
    plt.locator_params(axis='x', nbins=10)
    plt.plot(xraw, color_freq, 'bo-', alpha=0.3)
    plt.plot(xraw, color_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Date')
    plt.ylabel('Percent Frequency')
    plt.suptitle(major_color[i].capitalize())
    plt.savefig('./results/all_byday_' + major_color[i].replace(' ', '') + '.png')

# Clothing Category Over Time

### All Anchors

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
category_count = []
for i in range(0, len(clothing_category)):
    category_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    week_idx = calc_week_index(date[i])
    category = attributes[i][7]
    category_count[category][week_idx] += 1
    total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(clothing_category)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    category_freq = category_count[i] / total_count
    # smooth data
    category_smooth = savgol_filter(category_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, category_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, category_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle(clothing_category[i].capitalize())
    plt.savefig('./results/all_' + clothing_category[i].replace(' ', '') + '.png')

### Female

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
category_count = []
for i in range(0, len(clothing_category)):
    category_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if gender[i] == gender_dict['F']:
        week_idx = calc_week_index(date[i])
        category = attributes[i][7]
        category_count[category][week_idx] += 1
        total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(clothing_category)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    category_freq = category_count[i] / total_count
    # smooth data
    category_smooth = savgol_filter(category_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, category_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, category_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('Female - ' + clothing_category[i].capitalize())
    plt.savefig('./results/female_' + clothing_category[i].replace(' ', '') + '.png')

### Male

In [ ]:
# structures to hold color frequency info
total_count = np.zeros((total_weeks))
category_count = []
for i in range(0, len(clothing_category)):
    category_count.append(np.zeros((total_weeks)))

# count frequency of each color
for i in range(0, len(data)):
    if gender[i] == gender_dict['M']:
        week_idx = calc_week_index(date[i])
        category = attributes[i][7]
        category_count[category][week_idx] += 1
        total_count[week_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
for i in range(0, len(clothing_category)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    category_freq = category_count[i] / total_count
    # smooth data
    category_smooth = savgol_filter(category_freq, 15, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.plot(xdate_objs, category_freq, 'bo-', alpha=0.3)
    plt.plot(xdate_objs, category_smooth, 'r-')
    plt.axvline(x=year_dates[0], linestyle='--')
    plt.axvline(x=year_dates[1], linestyle='--')
    plt.xlabel('Year-Month')
    plt.ylabel('Percent Frequency')
    plt.suptitle('Male - ' + clothing_category[i].capitalize())
    plt.savefig('./results/male_' + clothing_category[i].replace(' ', '') + '.png')

# Glasses Over Time

In [ ]:
total_count = np.zeros((total_weeks))
glasses_count = np.zeros((total_weeks))

for i in range(0, len(data)):
    week_idx = calc_week_index(date[i])
    has_glasses = attributes[i][10]
    glasses_count[week_idx] += has_glasses
    total_count[week_idx] += has_glasses

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_weeks)

xdates = []
for i in range(0, xraw.shape[0]):
    cur_date = date_from_index(i)
    xdates.append(cur_date)
xdate_objs = [datetime.date(year, month, day) for (year, month, day) in xdates]
year_dates = [xdate_objs[48], xdate_objs[96]]

# find percentage and plot
nocount_inds = np.where(total_count == 0)[0]
total_count[nocount_inds] = sys.maxsize
glasses_freq = glasses_count / total_count
# smooth data
glasses_smooth = savgol_filter(glasses_freq, 15, 3)
# plot
plt.figure(figsize=(12, 4))
plt.plot(xdate_objs, category_freq, 'bo-', alpha=0.3)
plt.plot(xdate_objs, category_smooth, 'r-')
plt.axvline(x=year_dates[0], linestyle='--')
plt.axvline(x=year_dates[1], linestyle='--')
plt.xlabel('Year-Month')
plt.ylabel('Percent Frequency')
plt.suptitle(wearing_glasses[1].capitalize())
plt.savefig('./results/all_' + wearing_glasses[1].replace(' ', '') + '.png')

# Clothing Category Throughout Single Day

In [ ]:
min_year = 2015
min_month = 1
min_day = 1
months_per_year = 12
weeks_per_month = 4
days_per_month = 31
week_length = 1.*days_per_month / weeks_per_month

def calc_time_index(time):
    ''' Bin in half hours. '''
    hour, minute, _ = time
    idx = hour*2 + int(1.*minute / 30)
    return idx

def time_from_idx(idx):
    hour = int(1.*idx / 2)
    minute = (idx % 2)*30
    second = 0
    return (hour, minute, second)
        

total_half_hours = calc_time_index((23, 59, 59)) + 1
print(total_half_hours)
print(calc_time_index((0, 54, 0)))
print(calc_time_index((23, 36, 1)))

### Everyone all networks

In [ ]:
total_count = np.zeros((total_half_hours))
category_count = []
for i in range(0, len(clothing_category)):
    category_count.append(np.zeros((total_half_hours)))

# count frequency of each color
for i in range(0, len(data)):
    time_idx = calc_time_index(time[i])
    category = attributes[i][7]
    category_count[category][time_idx] += 1
    total_count[time_idx] += 1

In [ ]:
from scipy.signal import savgol_filter
import datetime

xraw = np.arange(0, total_half_hours)

xtimes = []
for i in range(0, xraw.shape[0]):
    if i % 4 == 0:
        cur_time = time_from_idx(i)
        xtimes.append(cur_time)
xtime_objs = [datetime.time(hour, minute, second).isoformat() for (hour, minute, second) in xtimes]

# find percentage and plot
for i in range(0, len(clothing_category)):
    nocount_inds = np.where(total_count == 0)[0]
    total_count[nocount_inds] = sys.maxsize
    category_freq = category_count[i] / total_count
    # smooth data
    category_smooth = savgol_filter(category_freq, 5, 3)
    # plot
    plt.figure(figsize=(12, 4))
    plt.xticks(xraw, xtime_objs)
    plt.locator_params(axis='x', nbins=12)
    plt.plot(xraw, category_freq, 'bo-', alpha=0.3)
    plt.plot(xraw, category_smooth, 'r-')
    plt.xlabel('Time (Eastern)')
    plt.ylabel('Percent Frequency')
    plt.suptitle(clothing_category[i].capitalize())
    plt.savefig('./results/all_day_' + clothing_category[i].replace(' ', '') + '.png')